# Support Vector Machine (SMV), Cesar Avellaneda, 2202214746

Esto corresponde al noteBook del algoritmo SMV, a cargo de Cesar Avellaneda:

Y contiene los siguientes elementos:
1. Cargue de los datos.

2. Entendimiento de los datos: Describir las características más relevantes de los datos y todo el perfilamiento de datos, incluir el análisis de calidad de datos y hacer una preselección de las variables más importantes para la etapa de modelado.

3. Preparación de datos: Solucionar los problemas de calidad de datos previamente identificados que afecten el modelo a construir. Además, debe aplicar todos los proceso de preprocesamiento de datos necesarios para la construcción del modelo de regresión.

4. Modelado: Utilizando las variables previamente seleccionadas, construir un modelo de regresión que estime la variable objetivo con el menor error posible.


# Entendimiento del Problema:

El problema principal radica en la necesidad de analizar de forma eficiente y automática grandes volúmenes de opiniones en lenguaje natural para relacionarlas con los ODS 3 (Salud y Bienestar), 4 (Educación de Calidad) y 5 (Igualdad de Género). Actualmente, este proceso es manual y requiere la intervención de expertos, lo que lo hace costoso y lento. La oportunidad consiste en desarrollar un modelo analítico automatizado que reduzca el tiempo y los recursos necesarios para este análisis, permitiendo una respuesta más rápida y efectiva a las necesidades de los ciudadanos.

# Enfoque Analitico:

Categoría de Análisis:
El análisis será predictivo y descriptivo. Descriptivo en el sentido de identificar y categorizar opiniones actuales, y predictivo al inferir tendencias y futuras necesidades o áreas de intervención.

Tipo y Tarea de Aprendizaje:
Aprendizaje Supervisado: Para clasificar las opiniones y relacionarlas con los ODS.

Técnicas y Algoritmos:
Modelos de Clasificación: SVM (Support Vector Machines).

# 1. Carga de los Datos:

In [ ]:
import inflect
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
import pandas as pd
import numpy as np
import sys
import re, string, unicodedata
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.base import BaseEstimator, ClassifierMixin
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
entrevistasODS = pd.read_excel('./data/ODScat_345.xlsx')

# 2. Perfilamiento y entendimiento de datos:

In [ ]:
print(entrevistasODS.head())

In [ ]:
entrevistasODS.info()

In [5]:
entrevistadosCopy = entrevistasODS.copy()
entrevistadosCopy['Conteo'] = [len(x) for x in entrevistadosCopy['Textos_espanol']]
#entrevistadosCopy['Moda'] =
entrevistadosCopy['Max'] = [[max([len(x) for x in i.split(' ')])][0] for i in entrevistadosCopy['Textos_espanol']]
entrevistadosCopy['Min'] = [[min([len(x) for x in i.split(' ')])][0] for i in entrevistadosCopy['Textos_espanol']]


In [ ]:
entrevistadosCopy

# 3. Limpieza de Datos:

In [7]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        if word is not None:
          new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
          new_words.append(new_word)
    return new_words

def to_lowercase(words):
    new_words=[]
    for word in words:
        w= word.lower()
        new_words.append(w)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        if word is not None:
            new_word = re.sub(r'[^\w\s]', '', word)
            if new_word != '':
                new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()    
    print(words)
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
            print("if " + new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    spanish_sw = set(stopwords.words('spanish'))
    new_words = []
    for word in words:
        if word not in spanish_sw:
            new_words.append(word)
    return new_words


def preprocessing(words):
    words = to_lowercase(words)
    words = replace_numbers(words)
    words = remove_punctuation(words)
    words = remove_non_ascii(words)
    words = remove_stopwords(words)
    return words

In [ ]:
entrevistasODS['palabras'] = entrevistasODS['Textos_espanol'].apply(word_tokenize)
entrevistasODS.head()

In [ ]:
entrevistasODS['palabras'].dropna()

In [ ]:
entrevistasODS['palabras'].info()

In [ ]:
entrevistasODS['palabras1']=entrevistasODS['palabras'].apply(preprocessing)
entrevistasODS.head()

In [ ]:
entrevistasODS['palabras'] = entrevistasODS['palabras'].apply(lambda x: ' '.join(map(str, x)))
entrevistasODS

In [ ]:
X_data, y_data = entrevistasODS['palabras'],entrevistasODS['sdg']
y_data

In [ ]:
dummy = CountVectorizer(binary=True)
X_dummy = dummy.fit_transform(X_data)
print(X_dummy.shape)
X_dummy.toarray()[0]

In [ ]:
count = CountVectorizer()
X_count = count.fit_transform(X_data)
print(X_count.shape)
X_count.toarray()[0]

# SMV

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_count, y_data, test_size=0.3, random_state=42)
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


# Bolsa de Palabras

In [17]:
# X_count = count.fit_transform(X_data)
# y_pred = svm_model.predict(X_test)
# print(classification_report(y_test, y_pred))
# print(confusion_matrix(y_test, y_pred))

# param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf', 'poly'], 'gamma': ['scale', 'auto']}
# grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2)
# grid.fit(X_train, y_train)
# print("Mejores parámetros:", grid.best_params_)
# y_pred = grid.predict(X_test)
# print(classification_report(y_test, y_pred))
# print(confusion_matrix(y_test, y_pred))

In [18]:
X_data, y_data = entrevistasODS['palabras'],entrevistasODS['sdg']


In [ ]:
dummy = CountVectorizer(binary=True)
X_dummy = dummy.fit_transform(X_data) 
print(X_dummy.shape)

X_dense = X_dummy.toarray()
X_df = pd.DataFrame(X_dense, columns=dummy.get_feature_names_out())
X_df


In [20]:
tt = TfidfTransformer(norm='l2', use_idf=True)
tt_matrix = tt.fit_transform(X_dummy)


In [ ]:
tt_matrix_dense = tt_matrix.toarray()
vocab = dummy.get_feature_names_out()
tt_df = pd.DataFrame(np.round(tt_matrix_dense, 2), columns=vocab)
tt_df


In [ ]:
import scipy.sparse as sp
feature_names = list(tt_df.columns)
# build the document frequency matrix
df = np.diff(sp.csc_matrix(tt_df, copy=True).indptr)
df = 1 + df # adding 1 to smoothen idf later
# show smoothened document frequencies
pd.DataFrame([df], columns=feature_names)

In [ ]:

# compute inverse document frequencies
total_docs = 1 + len(vocab)
idf = 1.0 + np.log(float(total_docs) / df)
# show smoothened idfs
pd.DataFrame([np.round(idf, 2)], columns=feature_names)

In [ ]:
# compute idf diagonal matrix
total_features = tt_df.shape[1]
idf_diag = sp.spdiags(idf, diags=0, m=total_features, n=total_features)
idf_dense = idf_diag.todense()
# print the idf diagonal matrix
pd.DataFrame(np.round(idf_dense, 2))

In [ ]:
# compute tfidf feature matrix
tf = np.array(tt_df, dtype="float64")
tfidf = tf * idf
# view raw tfidf feature matrix
tfidf = pd.DataFrame(np.round(tfidf, 2), columns=feature_names)
tfidf


In [28]:
X_train, X_test, y_train, y_test = train_test_split(tfidf, y_data, test_size=0.3, random_state=42)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Entrenamos el clasificador Naive Bayes con la matriz TF-IDF
clf = MultinomialNB()
clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Hacemos predicciones en el conjunto de prueba
y_pred = clf.predict(X_test)

# Calculamos la precisión
print(f"Precisión: {accuracy_score(y_test, y_pred):.2f}")

# Mostramos el reporte de clasificación
print(classification_report(y_test, y_pred))

# Árbol de clasificación

In [35]:
from sklearn.tree import DecisionTreeClassifier


# Entrenamos el clasificador Decision Tree con la matriz TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(entrevistasODS['palabras'])
Y=entrevistasODS['sdg']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
clf = DecisionTreeClassifier()

In [ ]:
clf.fit(X_train, y_train)

In [40]:
y_pred = clf.predict(X_test) 

In [41]:
cm = confusion_matrix(y_test, y_pred)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay


disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot(cmap=plt.cm.Blues)

plt.show()

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score


print('Exactitud: %.2f' % accuracy_score(y_test, y_pred))
print("Recall: {}".format(recall_score(y_test,y_pred, average='weighted')))
print("Precisión: {}".format(precision_score(y_test,y_pred, average='weighted')))
print("Puntuación F1: {}".format(f1_score(y_test,y_pred, average='weighted')))

In [ ]:
print(classification_report(y_test, y_pred))